## Parte 3
### Actividad 7
El modelo que se pensó ocupar es uno basado en el paper de [Weissenborn et al](https://arxiv.org/abs/1703.04816). Este modelo plantea un baseline simple para implementar una red neuronal que permita resolver el problema de Pregunta/Respuesta. De manera general, el modelo propuesto se muestra en la siguiente figura:
![Modelo propuesto para resolver el problema de Pregunta/Respuesta](https://i.imgur.com/KnnCaLS.png)

Para modelar los documentos de manera eficiente se decidió tomar el modelo Wor2Vec, para lo cual se crea un diccionario basado en todas las palabras disponibles en el set de parrafos y preguntas.

Después de crear el modelo Inicialmente se tiene dos modelos, uno que permita retener el conocimiento de los parrafos y otro modelo que sea para retener el conocimiento de las preguntas. La estructura de ambos modelos esta dada por:
- Una capa de encoding, que permite obtener un documento en su versión vectorial, donde cada palabra antes de ser ingresada a la red es reemplazada por un número entero que representa su posición en el vector de palabras.
- Una capa recurrente del tipo bidireccional para retener el conocimiento.

En este modelo se eligió una capa recurrente bidireccional debido a que esta entiende mucho mejor el contexto ya que debido a su diseño este puede retener la información respecto al [pasado y futuro](https://www.quora.com/When-should-one-use-bidirectional-LSTM-as-opposed-to-normal-LSTM). Esto permite mejorar de cierta manera el rendimiento de la red neuronal.

El siguiente paso es tener una capa de interacción que concatena las entradas de ambos modelos, esta capa de concatenación se le pasa a otra capa recurrente del tipo LSTM bidireccional y las salidas se envian a una capa densa que sirve de clasificador, el cual devuelve la posición del primer carácter de la respuesta en el párrafo.

Para el proceso de modelamiento de los documentos se decidió usar la librería Gensim que permite generar un modelo Word2Vec y ser añadido como la capa de encoding a una red neuronal hecha en Keras.

### Actividad 8
Esta red neuronal fue entrenada en una GPU Nvidia 1080Ti, haciendo uso de la librería Keras corriendo bajo el backend de tensorflow. Inicialmente se realiza la configuración básica del entorno, se importan las librerías básicas y necesarias para el proceso de entrenamiento y por último se agrega un pequeño segmento de código que se comunica con el backend en Tensorflow para evitar que la GPU sea llenada de golpe y se pida memoria "on-demand".

In [1]:
import json
import string
import re
import sys
import gensim
import gensim.parsing.preprocessing as p
import numpy as np
from collections import Counter
from keras.models import Model, load_model
from keras.layers import Dropout, Dense, Input, LSTM, Bidirectional, concatenate
from keras import backend as K

# Dynamic allocation of GPU memory
if K.backend() == 'tensorflow':
    import tensorflow as tf
    from keras.backend.tensorflow_backend import set_session

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

/home/pablo/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


En esta sección de código se procede a la inicialización del dataset. La implementación consiste en inicialmente leer el archivo en formato json y la posterior lectura de los parrafos, las preguntas y las respuestas. Para el caso de los parrafos y las preguntas se sigue un proceso de post procesamiento en el cual se elimina las puntuaciones y espacios que están demas. En este caso no se hace un proceso de stemming ya que se tiene la hipótesis de que la red necesita de más contexto al respecto de tiempos, pluraridad, etc.

In [2]:
# Dataset reading
with(open('train-v1.1.json')) as json_data:
    d = json.load(json_data)

dataset = d['data']
print(len(dataset))

context_list = []
question_list = []
answer_list = []
answer_start = []
answer_end = []

context_size = 600
question_size = 50

CUSTOM_FILTER = [
    lambda x: x.lower(), p.strip_tags, p.strip_punctuation,
    p.strip_multiple_whitespaces
]

for article in dataset:
    for paragraph in article['paragraphs']:
        context = p.preprocess_string(paragraph['context'], CUSTOM_FILTER)
        if len(context) > context_size:
            context_size = len(context)

        for qa in paragraph['qas']:
            question = p.preprocess_string(qa['question'], CUSTOM_FILTER)
            if len(question) > question_size:
                question_size = len(question)

            for answer in qa['answers']:
                context_list.append(context)
                question_list.append(question)
                answer_list.append(answer['text'])
                answer_start.append(answer['answer_start'])
                answer_end.append(answer['answer_start'] + len(answer['text']) - 1)

print("Finished reading and preprocessing documents")

442
Finished reading and preprocessing documents


El siguiente paso es entrenar el modelo Word2Vec, para esto se hace uso de la librería *Word2Vec*, este vector se crea con un tamaño de 200 para las features.

In [3]:
# Training Word2Vec model
print("Training W2V model")

modelW2V = gensim.models.Word2Vec(context_list + question_list, size=200, workers=8, min_count=1, iter=20)

print("Finished to train W2V model")

Training W2V model
Finished to train W2V model


In [4]:
modelW2V.wv.most_similar(positive=['dog'])

[('dogs', 0.4859297573566437),
 ('pet', 0.4292300343513489),
 ('groomers', 0.41012632846832275),
 ('poultry', 0.40378862619400024),
 ('caretakers', 0.4020272195339203),
 ('familiaris', 0.3992404341697693),
 ('chicken', 0.38795578479766846),
 ('breed', 0.38742366433143616),
 ('canis', 0.38202208280563354),
 ('wolf', 0.36578139662742615)]

Antes de poder entrenar el dataset, es necesario pasar las estructuras de datos armadas a un formato más legible para Keras, el cual son *Numpy arrays*. Se procede a crear los arreglos necesarios para los datos de entrada y salida que se ocupará en el entrenamiento.

In [5]:
# Prepare training dataset
print("Preparing training dataset")
context_array = np.zeros((len(context_list), context_size), dtype='int32')
question_array = np.zeros((len(question_list), question_size), dtype='int32')
start_array = np.zeros((len(answer_start),), dtype='int32')
end_array = np.zeros(len(answer_end,), dtype='int32')

for i in range(len(context_list)):
    for j in range(len(context_list[i])):
        context_array[i][j] = modelW2V.wv.vocab[context_list[i][j]].index

for i in range(len(question_list)):
    for j in range(len(question_list[i])):
        question_array[i][j] = modelW2V.wv.vocab[question_list[i][j]].index

for i in range(len(answer_start)):
    start_array[i] = answer_start[i]
    
for i in range(len(answer_start)):
    end_array[i] = answer_end[i]
    
max_start = np.amax(start_array)
max_end = np.amax(end_array)
print(max_start)

Preparing training dataset
3126


Para la creación del modelo en este caso se usa la API funcional de Keras, debido a que el modelo Secuencial que ofrece no tiene la flexibilidad necesaria para poder crear el modelo propuesto. Es por eso que se crea el modelo en base a tensores y la API funcional de modelos de Keras.
En esta parte, se crea el modelo que se encarga de aprender las características del contexto o párrafo.

In [6]:
# Context Model
context_input = Input(shape=(context_size, ), dtype='int32', name='context_input')
x = modelW2V.wv.get_keras_embedding()(context_input)
lstm_out = Bidirectional(LSTM(256, return_sequences=True), merge_mode='mul')(x)
drop1 = Dropout(0.5)(lstm_out)

Igualmente se crea el modelo que se encarga de aprender las características relacionadas a las preguntas.

In [7]:
# Question Model
question_input = Input(shape=(question_size,), dtype='int32', name='question_input')
x = modelW2V.wv.get_keras_embedding()(question_input)
lstm_out = Bidirectional(LSTM(256, return_sequences=True), merge_mode='mul')(x)
drop2 = Dropout(0.5)(lstm_out)

Por último se crea la capa de interacción, el cuál se encarga de unir los dos modelos anteriores, para esto se usa el método de concatenación. Seguido de esto se agrega otra capa LSTM bidireccional y las dos salidas que se necesitan en la red.

In [8]:
# Merge model
merge_layer = concatenate([drop1, drop2], axis=1)
lstm = Bidirectional(LSTM(512, return_sequences=False), merge_mode='mul')(merge_layer)
softmax1 = Dense(max_start, activation='softmax')(lstm)
softmax2 = Dense(max_end, activation='softmax')(lstm)

En este punto el modelo es creado y se lo compila. En este caso se usa la función de pérdida "Sparce Categorical CrossEntropy". Y por último, se genera un resumen de la estructura de la red.

In [9]:
model = Model(inputs=[context_input, question_input], outputs=[softmax1, softmax2])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context_input (InputLayer)      (None, 679)          0                                            
__________________________________________________________________________________________________
question_input (InputLayer)     (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 679, 200)     17265200    context_input[0][0]              
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 50, 200)      17265200    question_input[0][0]             
__________________________________________________________________________________________________
bidirectio

Se procede a la fase de entrenamiento y posterior guardado de los pesos en disco. Para la fase de entrenamiento, debido al tamaño del dataset, se deicidó realizar el entrenamiento por partes, siendo que para cada 30 iteraciones se hace un entrenamiento con una sección de 10000 datos del datase.

In [10]:
batch_size = 160
split = 10000

model_history = model.fit([context_array[:split], question_array[:split]],
                          [start_array[:split], end_array[:split]], verbose=1,
                          batch_size=batch_size, epochs=30)

Epoch 1/30
10000/10000 [==============================] - 162s 16ms/step - loss: 14.4568 - dense_1_loss: 7.1960 - dense_2_loss: 7.2608 - dense_1_acc: 0.0230 - dense_2_acc: 0.0019
Epoch 2/30
10000/10000 [==============================] - 160s 16ms/step - loss: 13.3812 - dense_1_loss: 6.6479 - dense_2_loss: 6.7333 - dense_1_acc: 0.0244 - dense_2_acc: 0.0061
Epoch 3/30
10000/10000 [==============================] - 160s 16ms/step - loss: 13.2717 - dense_1_loss: 6.5903 - dense_2_loss: 6.6814 - dense_1_acc: 0.0292 - dense_2_acc: 0.0064
Epoch 4/30
10000/10000 [==============================] - 160s 16ms/step - loss: 13.1375 - dense_1_loss: 6.5162 - dense_2_loss: 6.6213 - dense_1_acc: 0.0349 - dense_2_acc: 0.0067
Epoch 5/30
10000/10000 [==============================] - 160s 16ms/step - loss: 12.9636 - dense_1_loss: 6.4292 - dense_2_loss: 6.5344 - dense_1_acc: 0.0366 - dense_2_acc: 0.0088
Epoch 6/30
10000/10000 [==============================] - 160s 16ms/step - loss: 12.7677 - dense_1_loss: 

In [11]:
print('Saving models')
model.save('squad_model.h5')
modelW2V.save('w2vec_model.h5')
with open('history_squad.json', 'w') as outfile:
    json.dump(model_history.history, outfile)

Saving models


In [14]:
batch_size = 256
split = 20000

model_history = model.fit([context_array[:split], question_array[:split]],
                          [start_array[:split], end_array[:split]], verbose=1,
                          batch_size=batch_size, epochs=30)

Epoch 1/30
20000/20000 [==============================] - 294s 15ms/step - loss: 10.6324 - dense_1_loss: 5.2697 - dense_2_loss: 5.3627 - dense_1_acc: 0.0820 - dense_2_acc: 0.0659
Epoch 2/30
20000/20000 [==============================] - 293s 15ms/step - loss: 10.0700 - dense_1_loss: 4.9905 - dense_2_loss: 5.0795 - dense_1_acc: 0.0943 - dense_2_acc: 0.0791
Epoch 3/30
20000/20000 [==============================] - 293s 15ms/step - loss: 9.7762 - dense_1_loss: 4.8467 - dense_2_loss: 4.9295 - dense_1_acc: 0.1042 - dense_2_acc: 0.0916
Epoch 4/30
20000/20000 [==============================] - 293s 15ms/step - loss: 9.5593 - dense_1_loss: 4.7428 - dense_2_loss: 4.8165 - dense_1_acc: 0.1086 - dense_2_acc: 0.1001
Epoch 5/30
20000/20000 [==============================] - 293s 15ms/step - loss: 9.3806 - dense_1_loss: 4.6554 - dense_2_loss: 4.7252 - dense_1_acc: 0.1166 - dense_2_acc: 0.1064
Epoch 6/30
20000/20000 [==============================] - 293s 15ms/step - loss: 9.2246 - dense_1_loss: 4.57

In [16]:
print('Saving models')
model.save('squad_model.h5')
with open('history_squad2.json', 'w') as outfile:
    json.dump(model_history.history, outfile)
    
K.clear_session()

Saving models


Para terminar esta fase de entrenamiento, se guardan los datos del historial de entrenamiento y los pesos del Word2Vec y del modelo entrenado y se procede a limpiar la memoria para dejar todo en limpio e iniciar la fase de testing.

### Parte 9
Basandonos en el ejemplo de evaluación del dataset SQuAD se agregan las funciones que permitan evaluar y procesar los archivos de test y resultados de la predicción para así conseguir las métricas necesarias.

In [2]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(dataset, predictions):
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    message = 'Unanswered question ' + qa['id'] + \
                              ' will receive score 0.'
                    print(message, file=sys.stderr)
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

Ya con las funciones necesarias, se carga desde el disco los pesos del modelo entrenado y el bag of words. Se procede a cargar los párrafos y las preguntas del dataset de prueba, pasarlos a la estructura que el modelo pide y proceder a la fase de predicción, donde se almacenan las respuestas en un diccionario en formato JSON siguiendo el formato que SQuAD sugiere para la evaluación.

In [3]:
model = load_model('squad_model.h5')
modelW2V = gensim.models.Word2Vec.load("w2vec_model.h5")

with(open('dev-v1.1.json')) as json_data:
    d = json.load(json_data)

dataset = d['data']
print(len(dataset))

context_list = []
context_entire = []
question_list = []
question_entire = []
question_ids = []
answers = []

context_size = 679
question_size = 50

CUSTOM_FILTER = [
    lambda x: x.lower(), p.strip_tags, p.strip_punctuation,
    p.strip_multiple_whitespaces
]

for article in dataset:
    for paragraph in article['paragraphs']:
        context = p.preprocess_string(paragraph['context'], CUSTOM_FILTER)
        if len(context) > context_size:
            context_size = len(context)

        for qa in paragraph['qas']:
            question = p.preprocess_string(qa['question'], CUSTOM_FILTER)
            if len(question) > question_size:
                question_size = len(question)
            context_list.append(context)
            context_entire.append(paragraph['context'])
            question_list.append(question)
            question_entire.append(qa['question'])
            question_ids.append(qa['id'])

print("Finished reading and preprocessing documents")

context_array = np.zeros((len(context_list), context_size), dtype='int32')
question_array = np.zeros((len(question_list), question_size), dtype='int32')

for i in range(len(context_list)):
    for j in range(len(context_list[i])):
        try:
            context_array[i][j] = modelW2V.wv.vocab[context_list[i][j]].index
        except KeyError:
            context_array[i][j] = 0

for i in range(len(question_list)):
    for j in range(len(question_list[i])):
        try:
            question_array[i][j] = modelW2V.wv.vocab[question_list[i][j]].index
        except KeyError:
            question_array[i][j] = 0

print(context_array.shape)
print(question_array.shape)
predictions = model.predict([context_array, question_array], verbose=1)

starts = predictions[0]
ends = predictions[1]

response = {}
for i in range(len(starts)):
    response[question_ids[i]] = context_entire[i][np.argmax(starts[i]):np.argmax(ends[i])]

with open('train_result.json', 'w') as outfile:
    json.dump(response, outfile)

48
Finished reading and preprocessing documents
(10570, 679)
(10570, 50)
10570/10570 [==============================] - 143s 14ms/step


Finalmente se procede a la fase de evaluaación, donde se cargan los archivos de test junto a las respuestas obtenidas por el modelo y se llama a las funciones correspondientes para su evaluación:

In [4]:
expected_version = '1.1'
with open('dev-v1.1.json') as dataset_file:
    dataset_json = json.load(dataset_file)
    if dataset_json['version'] != expected_version:
        print('Evaluation expects v-' + expected_version +
              ', but got dataset with v-' + dataset_json['version'],
              file=sys.stderr)
    dataset = dataset_json['data']
with open('train_result.json') as prediction_file:
    predictions = json.load(prediction_file)
print(json.dumps(evaluate(dataset, predictions)))

{"exact_match": 0.30274361400189215, "f1": 2.2069435388678484}


Como se puede observar, el resultado obtenido es bastante pobre, hay varias suposiciones al respecto de porque sucede esto:
- Para empezar, en la fase de entrenamiento se llegó a un accuracy de un 17% aproximadamente comparandose con los propios datos de entrenmiento. Igualmente la función de perdida nunca dejo ser un valor alto, por lo tanto hace suponer que el modelo estaba muy lejos para converger, y por último, despues de varios epochs se observaba que el modelo empezaba a saturarse. Esto permite concluir que el modelo no estaba aprendiendo y lo poco que aprendia lo hacia de memoria.
- El modelo usado es muy probablemente que pueda ser mejorado agregando algunas capas ocultas más que permitan extraer más features del texto.
- Igualmente podría ser beneficioso hacer un mejor procesamiento del vocabulario. Tal vez haciendo que este sea mucho más rico en palabras.
- Es probable que el tipo de neuronas ocupadas en el modelo no eran las adecuadas o tal vez era necesario hacer un mayor tunning al respecto.

Como trabajo futuro se espera poder entrenar un mejor modelo para así lograr resolver el problema de Pregunta/Respuesta planteado